## LangChain with Simple Chain Pattern
> **Expected Output**: Action:

This code is setting up a LangChain pipeline that connects to a custom OpenAI-compatible large language model (LLM) endpoint (Qwen 3 Ultra) using credentials from the environment, then prompts it to suggest a company name for a given product.


In [2]:
import os
import requests
import json
from openai import OpenAI
import httpx
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
import warnings
from cfenv import AppEnv
import sys, os

# go one level up from cflangchainfolder/ to project root
sys.path.append(os.path.abspath(".."))
from cfutils import CFGenAIService

warnings.filterwarnings('ignore')

# Load CF environment
env = AppEnv()

# load your service details replace name with your Gen AI service.  Gen AI service is bound to the app
genai = CFGenAIService("gpt-oss:120b")
print("API Base:", genai.api_base)
print("Config URL:", genai.config_url)

# List available models
models = genai.list_models()
for m in models:
    print(f"- {m['name']} (capabilities: {', '.join(m['capabilities'])})")

# construct chat_credentials
chat_credentials = {
    "api_base": genai.api_base,
    "api_key": genai.api_key,
    "model_name": models[0]["name"]
}

# 2. HTTP client (optional but recommended for custom config)
httpx_client = httpx.Client(verify=False)  # verify=False if your endpoint needs --insecure

# 3. Initialize the LLM
llm = ChatOpenAI(
    temperature=0.9,
    model=chat_credentials["model_name"],   # model name from CF service
    base_url=chat_credentials["api_base"],  # OpenAI-compatible endpoint
    api_key=chat_credentials["api_key"],    # Bearer token
    http_client=httpx_client
)

prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

chain = LLMChain(llm=llm, prompt=prompt)
product = "queen-sized sheet sets"
response = chain.run(product)
print(response)

API Base: https://genai-proxy.sys.tas-ndc.kuhn-labs.com/tanzu-gpt-oss-120b-b83126a/openai
Config URL: https://genai-proxy.sys.tas-ndc.kuhn-labs.com/tanzu-gpt-oss-120b-b83126a/config/v1/endpoint
- gpt-oss:120b (capabilities: CHAT, TOOLS)
### Quick‑Answer
If you need a **single, punchy name that instantly tells customers “we make queen‑size sheet sets”**, something like **“Queen‑Fit Linens”** or **“Queen‑Size Comfort Co.”** does the job in just two words.

---

## Why a “descriptive‑plus‑brandable” name works best

| Goal | What it looks like in a name | Why it matters |
|------|-----------------------------|----------------|
| **Clarity** | “Queen‑Size” or “Queen‑Fit” | Customers (and search engines) instantly know what you sell. |
| **Memorability** | Short, one‑or‑two‑syllable words | Easy to recall, repeat, and recommend. |
| **Brand Personality** | “Comfort”, “Royal”, “Cozy”, “Dream” | Gives an emotional hook beyond the product description. |
| **Scalability** | Leaves room to add o